## Import Libraries

In [2]:
!pip install pmdarima


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 9.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from pmdarima import auto_arima

from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

## Data Loaded

In [31]:
df = pd.read_csv("demand_data.csv", parse_dates=["week"])

# Extract numeric ids for correct sorting
df["store_num"] = df["store"].str.extract(r"(\d+)").astype(int)
df["product_num"] = df["product"].str.extract(r"(\d+)").astype(int)

# Correct sorting
df = df.sort_values(["store_num", "product_num", "week"])

# Drop helper columns
df = df.drop(columns=["store_num", "product_num"])


In [32]:
df

,week,store,product,demand
0,2024-01-07,store_1,product_1,69
1,2024-01-14,store_1,product_1,64
2,2024-01-21,store_1,product_1,91
3,2024-01-28,store_1,product_1,95
4,2024-02-04,store_1,product_1,73
...,...,...,...,...
19995,2025-11-02,store_10,product_20,168
19996,2025-11-09,store_10,product_20,143
19997,2025-11-16,store_10,product_20,131
19998,2025-11-23,store_10,product_20,145


## SARIMA Parameters estimation

In [33]:
results = []

for (store, product), g in df.groupby(["store", "product"]):

    series = g["demand"].values

    # ---- Minimum length safeguard ----
    if len(series) < 20:
        continue

    try:
        model = auto_arima(
            series,
            seasonal=True,
            m=4,                       # 4-week seasonality
            d=None,                    # auto detect
            D=None,                    # auto detect
            start_p=0, start_q=0,
            max_p=3, max_q=3,
            start_P=0, start_Q=0,
            max_P=2, max_Q=2,
            with_intercept=True,
            trace=True,
            error_action="ignore",
            suppress_warnings=True,
            stepwise=True
        )

        order = model.order                 # (p, d, q)
        seasonal_order = model.seasonal_order  # (P, D, Q, s)

        results.append({
            "store": store,
            "product": product,
            "p": order[0],
            "d": order[1],
            "q": order[2],
            "P": seasonal_order[0],
            "D": seasonal_order[1],
            "Q": seasonal_order[2],
            "s": seasonal_order[3],
            "AIC": model.aic()
        })

    except Exception as e:
        print(f"Model failed for {store}-{product}: {e}")




Streaming output truncated to the last 5000 lines.
 ARIMA(3,1,0)(1,0,1)[4] intercept   : AIC=624.718, Time=0.62 sec
 ARIMA(3,1,1)(0,0,2)[4] intercept   : AIC=623.408, Time=0.91 sec
 ARIMA(3,1,1)(0,0,1)[4] intercept   : AIC=624.871, Time=0.55 sec
 ARIMA(3,1,1)(1,0,2)[4] intercept   : AIC=625.566, Time=1.09 sec
 ARIMA(3,1,1)(1,0,1)[4] intercept   : AIC=624.336, Time=0.68 sec
 ARIMA(2,1,1)(0,0,2)[4] intercept   : AIC=inf, Time=0.73 sec
 ARIMA(3,1,2)(0,0,2)[4] intercept   : AIC=624.876, Time=2.73 sec
 ARIMA(2,1,2)(0,0,2)[4] intercept   : AIC=inf, Time=1.27 sec
 ARIMA(3,1,1)(0,0,2)[4]             : AIC=690.502, Time=0.26 sec

Best model:  ARIMA(3,1,1)(0,0,2)[4] intercept
Total fit time: 19.405 seconds
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[4] intercept   : AIC=626.117, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[4] intercept   : AIC=608.153, Time=0.14 sec
 ARIMA(0,0,1)(0,1,1)[4] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,0,0)(0,1,0)[4]             : AIC=631.523, Time=0.01

In [35]:
results

[{'store': 'store_1',
  'product': 'product_1',
  'p': 0,
  'd': 0,
  'q': 0,
  'P': 2,
  'D': 1,
  'Q': 0,
  's': 4,
  'AIC': np.float64(619.8393581951493)},
 {'store': 'store_1',
  'product': 'product_10',
  'p': 1,
  'd': 0,
  'q': 0,
  'P': 2,
  'D': 1,
  'Q': 1,
  's': 4,
  'AIC': np.float64(401.28770992442514)},
 {'store': 'store_1',
  'product': 'product_11',
  'p': 0,
  'd': 0,
  'q': 0,
  'P': 2,
  'D': 1,
  'Q': 0,
  's': 4,
  'AIC': np.float64(411.2924561607814)},
 {'store': 'store_1',
  'product': 'product_12',
  'p': 0,
  'd': 0,
  'q': 0,
  'P': 2,
  'D': 1,
  'Q': 0,
  's': 4,
  'AIC': np.float64(411.80178660826596)},
 {'store': 'store_1',
  'product': 'product_13',
  'p': 3,
  'd': 1,
  'q': 0,
  'P': 2,
  'D': 0,
  'Q': 1,
  's': 4,
  'AIC': np.float64(659.0881513758893)},
 {'store': 'store_1',
  'product': 'product_14',
  'p': 0,
  'd': 0,
  'q': 0,
  'P': 2,
  'D': 1,
  'Q': 0,
  's': 4,
  'AIC': np.float64(413.78243893854426)},
 {'store': 'store_1',
  'product': 'pr

In [36]:
sarima_params = pd.DataFrame(results)
sarima_params


,store,product,p,d,q,P,D,Q,s,AIC
0,store_1,product_1,0,0,0,2,1,0,4,619.839358
1,store_1,product_10,1,0,0,2,1,1,4,401.287710
2,store_1,product_11,0,0,0,2,1,0,4,411.292456
3,store_1,product_12,0,0,0,2,1,0,4,411.801787
4,store_1,product_13,3,1,0,2,0,1,4,659.088151
...,...,...,...,...,...,...,...,...,...,...
195,store_9,product_5,3,1,0,2,0,1,4,623.282571
196,store_9,product_6,1,0,1,2,1,0,4,545.840701
197,store_9,product_7,3,1,0,2,0,1,4,625.265351
198,store_9,product_8,3,1,0,1,0,1,4,641.316677


In [37]:
# Extract numeric parts
sarima_params["store_num"] = sarima_params["store"].str.extract(r"(\d+)").astype(int)
sarima_params["product_num"] = sarima_params["product"].str.extract(r"(\d+)").astype(int)

# Sort numerically
sarima_params = sarima_params.sort_values(
    ["store_num", "product_num"]
).reset_index(drop=True)

# Drop helper columns
sarima_params = sarima_params.drop(columns=["store_num", "product_num"])


In [38]:
sarima_params

,store,product,p,d,q,P,D,Q,s,AIC
0,store_1,product_1,0,0,0,2,1,0,4,619.839358
1,store_1,product_2,3,1,0,1,0,2,4,613.373889
2,store_1,product_3,3,1,0,1,0,1,4,633.360471
3,store_1,product_4,2,1,1,0,0,0,4,648.748650
4,store_1,product_5,2,1,0,1,0,1,4,650.334450
...,...,...,...,...,...,...,...,...,...,...
195,store_10,product_16,0,0,0,2,1,0,4,440.626786
196,store_10,product_17,0,0,0,0,1,1,4,396.836732
197,store_10,product_18,3,1,1,0,0,2,4,623.407665
198,store_10,product_19,1,0,0,2,1,0,4,606.087323


In [39]:
sarima_params.to_csv("sarima_params.csv", index=False)

## Forecasting

In [4]:
df = pd.read_csv("demand_data.csv", parse_dates=["week"])
params_df = pd.read_csv("sarima_params.csv")

# Extract numeric parts
df["store_num"] = df["store"].str.extract(r"(\d+)").astype(int)
df["product_num"] = df["product"].str.extract(r"(\d+)").astype(int)

# Correct numeric sorting
df = df.sort_values(["store_num", "product_num", "week"])

df = df.drop(columns=["store_num", "product_num"])

forecast_results = []

# -----------------------------
# Forecast per store-product
# -----------------------------
for _, row in params_df.iterrows():

    store = row["store"]
    product = row["product"]

    p, d, q = int(row["p"]), int(row["d"]), int(row["q"])
    P, D, Q, s = int(row["P"]), int(row["D"]), int(row["Q"]), int(row["s"])

    g = df[(df["store"] == store) & (df["product"] == product)]

    # Ensure full 100 weeks exist
    if len(g) < 100:
        continue

    # Train: weeks 1–90
    train = g.iloc[:90]["demand"].values

    try:
        model = SARIMAX(
            train,
            order=(p, d, q),
            seasonal_order=(P, D, Q, s),
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        res = model.fit(disp=False)

        # Forecast weeks 91–100
        forecast = res.forecast(steps=10)

        forecast_weeks = g.iloc[90:100]["week"].values

        for week, value in zip(forecast_weeks, forecast):
            forecast_results.append({
                "store": store,
                "product": product,
                "week": week,
                "forecast_demand": max(0, int(round(value)))
            })

    except Exception as e:
        print(f"Forecast failed for {store}-{product}: {e}")


forecast_df = pd.DataFrame(forecast_results)

forecast_df.head()


,store,product,week,forecast_demand
0,store_1,product_1,2025-09-28,227
1,store_1,product_1,2025-10-05,222
2,store_1,product_1,2025-10-12,198
3,store_1,product_1,2025-10-19,190
4,store_1,product_1,2025-10-26,235


In [5]:
forecast_df

,store,product,week,forecast_demand
0,store_1,product_1,2025-09-28,227
1,store_1,product_1,2025-10-05,222
2,store_1,product_1,2025-10-12,198
3,store_1,product_1,2025-10-19,190
4,store_1,product_1,2025-10-26,235
...,...,...,...,...
1995,store_9,product_9,2025-11-02,201
1996,store_9,product_9,2025-11-09,190
1997,store_9,product_9,2025-11-16,194
1998,store_9,product_9,2025-11-23,200


In [6]:
forecast_df.to_csv("sarima_forecast.csv", index=False)

In [7]:
# Actual demand data (weeks 91–100 only)
actual_df = df[["store", "product", "week", "demand"]].rename(
    columns={"demand": "actual_demand"}
)


In [8]:
forecast_with_actual = forecast_df.merge(
    actual_df,
    on=["store", "product", "week"],
    how="left"
)


In [9]:
forecast_with_actual

,store,product,week,forecast_demand,actual_demand
0,store_1,product_1,2025-09-28,227,212
1,store_1,product_1,2025-10-05,222,216
2,store_1,product_1,2025-10-12,198,201
3,store_1,product_1,2025-10-19,190,193
4,store_1,product_1,2025-10-26,235,219
...,...,...,...,...,...
1995,store_9,product_9,2025-11-02,201,211
1996,store_9,product_9,2025-11-09,190,204
1997,store_9,product_9,2025-11-16,194,195
1998,store_9,product_9,2025-11-23,200,212


## Accuracy

In [10]:
df_eval = forecast_with_actual.copy()
df_eval = df_eval[df_eval["actual_demand"] > 0]

df_eval["APE"] = (
    np.abs(df_eval["actual_demand"] - df_eval["forecast_demand"])
    / df_eval["actual_demand"]
)

MAPE = df_eval["APE"].mean() * 100
print(f"MAPE: {MAPE:.2f}%")


MAPE: 7.58%


In [11]:
accuracy = 100 - MAPE
print(f"Forecast Accuracy: {accuracy:.2f}%")


Forecast Accuracy: 92.42%
